# Tableau Project - Videogame Co.

## Where do we open our new office?

### Instrucciones
#### Somos una compañía de Videogames de 50 empleados. Tenemos que abrir una oficina nueva, pero, ¿dónde?
#### Contamos con 20 desarrolladores, 20 UX/UI y 10 ejecutivos.
#### Tiene que haber alrededor:
#### 1-. Compañías de Software
#### 2-. Buen ratio entre grades compañías y start-ups
#### 3-. Empresas de diseño, diseño web, relacionado con UX
#### 4-. Restaurantes para que los ejecutivos puedan reunirse y hacer negocios 
#### 5-. Evitar compañías muy viejas alrededor

### Variables interesantes que mirar
#### Total money raised (inversiones), competitors, latitud y longitud de oficinas, tweeter (influencia de la empresa, API tweety), cathegory code